In [1]:
# Advanced Voice Assistant - Google Colab (Final v2.2 - Fixed Commands, Sanitized TTS, Safe Eval, Inline CSS)
# NOTE: Run in Google Colab. First cell installs dependencies.

# -------------------------
# INSTALL (run once)
# -------------------------
!pip install -q gradio
!pip install -q SpeechRecognition
!pip install -q gTTS
!pip install -q requests
!pip install -q python-dateutil
!pip install -q wikipedia
!apt-get install -y -qq espeak ffmpeg

# -------------------------
# IMPORTS
# -------------------------
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import requests
from datetime import datetime
import os
import re
import wikipedia
import tempfile
import warnings
import ast
import operator as op
import html
from typing import Tuple, Optional

warnings.filterwarnings('ignore')

# -------------------------
# CONFIG
# -------------------------
class VoiceAssistantConfig:
    """Configuration for Voice Assistant"""
    # Using your key directly (for demo). In real apps, keep this secret.
    OPENWEATHER_API_KEY = "9be813d77c5496f42683c9f2966a9e89"
    ASSISTANT_NAME = "Nova"
    VERSION = "2.2"
    DEFAULT_LOCATION = "London"
    HELP_TEXT = """
**Available Commands:**

🎤 **Greetings**: "Hello", "Hello Nova", "Hey"
🕐 **Time**: "What time is it?", "Current time"
📅 **Date**: "What's the date?", "What is today's date?"
🌤️ **Weather**: "What's the weather in Tokyo?", "Weather in London"
📚 **Knowledge**: "Tell me about artificial intelligence", "Who is Alan Turing?"
🔍 **Search**: "Search for Python tutorials"
🧮 **Calculate**: "Calculate 156 divided by 12", "What is 25 * 4?"
⏰ **Reminder**: "Remind me to finish the project"
ℹ️ **About Nova**: "What is your name?", "Who are you?", "What can you do?"
❓ **Help**: "Help", "Commands"
👋 **Exit**: "Goodbye", "Bye"
"""

# -------------------------
# SAFE EVAL (for calculations)
# -------------------------
_allowed_operators = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
    ast.USub: op.neg,
    ast.Mod: op.mod,
    ast.FloorDiv: op.floordiv,
    ast.UAdd: op.pos,
}

def safe_eval(expr: str):
    """Safely evaluate arithmetic expression using AST."""
    def _eval(node):
        if isinstance(node, ast.Constant):  # Python 3.8+
            if isinstance(node.value, (int, float)):
                return node.value
            raise ValueError("Only numbers are allowed")
        if isinstance(node, ast.Num):  # older AST
            return node.n
        if isinstance(node, ast.BinOp):
            op_type = type(node.op)
            if op_type not in _allowed_operators:
                raise ValueError("Operator not allowed")
            return _allowed_operators[op_type](_eval(node.left), _eval(node.right))
        if isinstance(node, ast.UnaryOp):
            op_type = type(node.op)
            if op_type not in _allowed_operators:
                raise ValueError("Operator not allowed")
            return _allowed_operators[op_type](_eval(node.operand))
        raise ValueError(f"Unsupported expression: {node!r}")

    parsed = ast.parse(expr, mode='eval')
    return _eval(parsed.body)

# -------------------------
# TTS SANITIZER
# -------------------------
def sanitize_for_tts(text: str, max_length: int = 1500) -> str:
    """
    Clean display text into a natural sentence string for TTS.
    Removes emojis, markdown tokens, inline links, weird symbols, and makes newlines into sentences.
    """
    if not text:
        return ""

    # Unescape HTML entities
    text = html.unescape(text)

    # Remove markdown characters and heavy visual tokens
    text = re.sub(r'(\*{1,2}|__|`|~~|#+|\[|\])', '', text)

    # Remove URLs entirely
    text = re.sub(r'https?:\/\/\S+|www\.\S+', '', text)

    # Remove unicode emoji / pictograph ranges
    emoji_pattern = re.compile(
        "["
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F680-\U0001F6FF"  # transport & map
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002700-\U000027BF"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub("", text)

    # Remove a set of stray icon characters often used in UIs
    text = re.sub(r'[•▪►→←▲▼★✦✶✳✔✖➜➔➤○●■◆]', ' ', text)

    # Convert newlines and pipes into sentence breaks
    text = re.sub(r'[\r\n]+', '. ', text)
    text = re.sub(r'\s*\|\s*', '. ', text)

    # Collapse multiple punctuation and spaces
    text = re.sub(r'\.{2,}', '. ', text)
    text = re.sub(r'\s+', ' ', text)

    # Trim leading/trailing punctuation/spaces
    text = text.strip()
    text = re.sub(r'^[\.\,\-\s]+', '', text)
    text = re.sub(r'[\.\,\-\s]+$', '', text)

    # Shorten overly long TTS text (try to cut at a sentence boundary)
    if len(text) > max_length:
        cut = text.rfind('.', 0, max_length)
        if cut == -1:
            text = text[:max_length].rstrip() + " ... continued."
        else:
            text = text[:cut].rstrip() + " ... continued."

    # Final safety: ensure it's printable
    text = text.encode('ascii', 'ignore').decode('ascii', 'ignore').strip()
    return text

# -------------------------
# ASSISTANT CLASS
# -------------------------
class AdvancedVoiceAssistant:
    def __init__(self):
        self.config = VoiceAssistantConfig()
        self.recognizer = sr.Recognizer()
        self.conversation_history = []
        self.session_stats = {
            'commands_processed': 0,
            'successful_responses': 0,
            'errors': 0
        }

    # TTS with sanitizer
    def speak(self, text: str) -> Tuple[str, Optional[str]]:
        """Return display text unchanged and a TTS audio file path created from sanitized text."""
        try:
            tts_text = sanitize_for_tts(text)
            if not tts_text:
                tts_text = "Sorry, I cannot read that out loud."

            # Capitalize first character for natural speech
            if len(tts_text) > 1:
                tts_text = tts_text[0].upper() + tts_text[1:]

            # create temp mp3 for Gradio
            tts = gTTS(text=tts_text, lang='en', slow=False)
            with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as fp:
                audio_file = fp.name
                tts.save(audio_file)

            self.session_stats['successful_responses'] += 1
            return text, audio_file
        except Exception as e:
            self.session_stats['errors'] += 1
            return f"Speech synthesis error: {str(e)}", None

    def recognize_speech_from_file(self, audio_file) -> str:
        """SpeechFile -> text using Google's recognizer."""
        try:
            with sr.AudioFile(audio_file) as source:
                self.recognizer.adjust_for_ambient_noise(source, duration=0.5)
                audio_data = self.recognizer.record(source)
                text = self.recognizer.recognize_google(audio_data)
                return text
        except sr.UnknownValueError:
            return "❌ Sorry, I couldn't understand the audio. Please speak clearly."
        except sr.RequestError as e:
            return f"❌ Speech recognition service error: {str(e)}"
        except Exception as e:
            return f"❌ Error processing audio: {str(e)}"

    def get_time(self) -> str:
        now = datetime.now()
        time_str = now.strftime('%I:%M %p')
        return f"🕐 The current time is **{time_str}**"

    def get_date(self) -> str:
        now = datetime.now()
        date_str = now.strftime('%A, %B %d, %Y')
        return f"📅 Today is **{date_str}**"

    def get_weather(self, city: str = None) -> str:
        if not city:
            city = self.config.DEFAULT_LOCATION

        api_key = self.config.OPENWEATHER_API_KEY
        if not api_key:
            return (
                "🌤️ **Weather Service Setup Required**\n\n"
                "No API key configured."
            )

        # sanitize city string: keep letters, spaces, hyphens
        city = re.sub(r'[^A-Za-z\s\-]', '', (city or "")).strip()
        if not city:
            city = self.config.DEFAULT_LOCATION

        try:
            url = "http://api.openweathermap.org/data/2.5/weather"
            params = {"q": city, "appid": api_key, "units": "metric"}
            response = requests.get(url, params=params, timeout=6)
            data = response.json()
            if response.status_code == 200:
                temp = data['main']['temp']
                feels_like = data['main']['feels_like']
                description = data['weather'][0]['description'].title()
                humidity = data['main']['humidity']
                wind_speed = data['wind']['speed']
                desc_l = description.lower()
                weather_emoji = "☀️" if "clear" in desc_l else "☁️" if "cloud" in desc_l else "🌧️" if "rain" in desc_l else "🌤️"
                return (
                    f"🌤️ **Weather in {city.title()}**\n\n"
                    f"{weather_emoji} **Condition**: {description}\n"
                    f"🌡️ **Temperature**: {temp}°C (Feels like {feels_like}°C)\n"
                    f"💧 **Humidity**: {humidity}%\n"
                    f"💨 **Wind Speed**: {wind_speed} m/s"
                )
            else:
                return f"❌ Couldn't fetch weather for '{city}'. {data.get('message','')}"
        except requests.RequestException:
            return "❌ Network error. Please check your internet connection."
        except Exception as e:
            return f"❌ Error: {str(e)}"

    def search_wikipedia(self, query: str) -> str:
        try:
            result = wikipedia.summary(query, sentences=3, auto_suggest=True)
            page = wikipedia.page(query, auto_suggest=True)
            return (
                f"📚 **Wikipedia: {page.title}**\n\n"
                f"{result}\n\n"
                f"🔗 Read more: {page.url}"
            )
        except wikipedia.exceptions.DisambiguationError as e:
            options = ', '.join(e.options[:5])
            return f"📚 Multiple results found for '{query}'. Try: {options}"
        except wikipedia.exceptions.PageError:
            return f"❌ No Wikipedia page found for '{query}'."
        except Exception as e:
            return f"❌ Wikipedia search error: {str(e)}"

    def web_search(self, query: str) -> str:
        return (
            f"🔍 **Search Results for: \"{query}\"**\n\n"
            "- (Demo) Integrate with Google/Bing/Custom Search APIs for real results.\n"
            "💡 Try: 'Tell me about ...' for quick info or use Wikipedia command."
        )

    def normalize_math_expression(self, expression: str) -> str:
        """
        Convert simple English math phrases to symbols.
        e.g. "156 divided by 12" -> "156 / 12"
        """
        expr = expression.lower()
        # word → operator replacements (do this before stripping)
        replacements = [
            ("divided by", "/"),
            ("multiplied by", "*"),
            ("times", "*"),
            ("x", "*"),
            ("plus", "+"),
            ("minus", "-"),
            ("modulo", "%"),
            ("remainder", "%"),
            ("mod", "%"),
        ]
        for word, op_sym in replacements:
            expr = expr.replace(word, f" {op_sym} ")

        # remove non-math/number characters
        expr = re.sub(r'[^0-9+\-*/().%\s]', ' ', expr)
        expr = re.sub(r'\s+', ' ', expr).strip()
        return expr

    def calculate(self, expression: str) -> str:
        try:
            # handle natural language math
            expression = self.normalize_math_expression(expression)
            if not expression:
                return "❌ Please provide a valid mathematical expression."
            result = safe_eval(expression)
            return f"🧮 **Calculation**\n\n**Expression**: {expression}\n**Result**: **{result}**"
        except ZeroDivisionError:
            return "❌ Error: Cannot divide by zero."
        except Exception:
            return "❌ Invalid calculation. Try: '25 * 4 + 10'"

    def set_reminder(self, reminder_text: str) -> str:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        return (
            f"⏰ **Reminder Set**\n\n📝 **Note**: {reminder_text}\n"
            f"🕐 **Created**: {timestamp}\n\n"
            "💡 In production, this would integrate with Google Calendar or system notifications."
        )

    def get_help(self) -> str:
        return self.config.HELP_TEXT

    def get_intro(self) -> str:
        return (
            f"👋 Hi, I'm **{self.config.ASSISTANT_NAME}**, your AI voice assistant (v{self.config.VERSION}).\n\n"
            "I can tell you the time, date, weather, answer questions, do calculations, set simple reminders, "
            "and more. Type **'help'** to see all available commands."
        )

    def process_command(self, text: str) -> str:
        self.session_stats['commands_processed'] += 1

        original_text = text.strip()
        if not original_text:
            return "❌ Please enter a command."

        # normalize
        text = original_text.lower().strip()

        # remove assistant name so "hello nova" still works naturally
        text = text.replace(self.config.ASSISTANT_NAME.lower(), "").strip()

        # ---- Help / Commands ----
        if "help" in text or "command" in text or "what can you do" in text:
            return self.get_help()

        # ---- Exit early if user says bye ----
        if any(word in text for word in ["exit", "quit", "goodbye", "bye", "see you"]):
            return (
                f"👋 **Goodbye!**\n\nThanks for using {self.config.ASSISTANT_NAME}!\n\n"
                f"**Session Stats:**\n- Commands processed: {self.session_stats['commands_processed']}\n"
                f"- Successful responses: {self.session_stats['successful_responses']}\n"
                f"- Errors: {self.session_stats['errors']}\n\nHave a great day! 🌟"
            )

        # ---- About / Intro ----
        if "your name" in text or "who are you" in text or "what are you" in text:
            return self.get_intro()

        # ---- Greeting ----
        if any(word in text for word in ["hello", "hi", "hey", "good morning", "good afternoon", "good evening"]):
            h = datetime.now().hour
            if h < 12:
                time_greeting = "Good morning"
            elif h < 18:
                time_greeting = "Good afternoon"
            else:
                time_greeting = "Good evening"
            return (
                f"👋 **{time_greeting}!**\n\n"
                f"I'm **{self.config.ASSISTANT_NAME}**, your AI voice assistant (v{self.config.VERSION}).\n\n"
                "How can I help you today? Type **'help'** to see available commands."
            )

        # ---- Time ----
        if "time" in text or "current time" in text or "what time" in text:
            return self.get_time()

        # ---- Date ----
        if "date" in text or "today's date" in text or "what is today" in text:
            return self.get_date()

        # ---- Weather ----
        if any(k in text for k in ["weather", "temperature", "forecast"]):
            city_match = re.search(r'in\s+([A-Za-z\-\s]+)', text)
            city = None
            if city_match:
                city = city_match.group(1).strip()
            else:
                parts = text.split()
                if len(parts) > 1:
                    maybe = parts[-1]
                    if maybe.isalpha():
                        city = maybe
            return self.get_weather(city)

        # ---- Reminder (before math, because it may contain numbers) ----
        if "remind" in text:
            reminder = text
            for phrase in ["remind me to", "remind me about", "remind me", "set reminder to", "set reminder"]:
                reminder = reminder.replace(phrase, "")
            reminder = reminder.strip()
            if not reminder:
                return "❌ Please tell me what you'd like to be reminded about."
            return self.set_reminder(reminder)

        # ---- Knowledge / Wikipedia ----
        if any(phrase in text for phrase in ["wikipedia", "tell me about", "who is", "what is"]):
            query = text
            for phrase in ["wikipedia", "tell me about", "who is", "what is"]:
                query = query.replace(phrase, "")
            query = query.strip()
            if not query:
                return "❌ Please specify what you'd like to know about."

            # If query looks like a math expression, treat as calculation instead
            if re.search(r'[\d\+\-*/%]', query):
                return self.calculate(query)

            return self.search_wikipedia(query)

        # ---- Calculate (words or symbols) ----
        if (
            any(word in text for word in ["calculate", "compute", "solve"]) or
            re.search(r'[\d\+\-*/%]', text) or
            any(word in text for word in ["divided by", "multiplied by", "times", "plus", "minus", "mod", "modulo"])
        ):
            expression = text
            for word in ["calculate", "compute", "solve", "what is", "what's", "please", "can you"]:
                expression = expression.replace(word, "")
            expression = expression.strip()
            return self.calculate(expression)

        # ---- Search ----
        if any(word in text for word in ["search", "google", "look up", "find"]):
            query = text
            for word in ["search for", "search", "google", "look up", "find"]:
                query = query.replace(word, "")
            query = query.strip()
            if not query:
                return "❌ Please specify what you'd like to search for."
            return self.web_search(query)

        # ---- Unknown ----
        return (
            "🤔 **I'm not sure how to help with that.**\n\n"
            "💡 Try asking:\n"
            "- 'What time is it?'\n"
            "- 'What's the weather in Tokyo?'\n"
            "- 'Tell me about Python'\n"
            "- 'Calculate 156 divided by 12'\n"
            "- Type 'help' for all commands"
        )

# -------------------------
# GRADIO UI (with inline CSS injection)
# -------------------------
def create_gradio_interface():
    assistant = AdvancedVoiceAssistant()

    def process_text(text: str) -> Tuple[str, Optional[str], str]:
        if not text or not text.strip():
            return "❌ Please enter a command.", None, ""
        timestamp = datetime.now().strftime('%H:%M:%S')
        assistant.conversation_history.append(f"[{timestamp}] You: {text}")
        response = assistant.process_command(text)
        assistant.conversation_history.append(f"[{timestamp}] {assistant.config.ASSISTANT_NAME}: {response}")
        _, audio = assistant.speak(response)
        history = "\n\n".join(assistant.conversation_history[-8:])
        return response, audio, history

    def process_audio(audio) -> Tuple[str, str, Optional[str], str]:
        if audio is None:
            return "❌ Please upload or record an audio file.", "", None, ""
        recognized = assistant.recognize_speech_from_file(audio)
        if recognized.startswith("❌"):
            return recognized, recognized, None, ""
        response, audio_out, history = process_text(recognized)
        return response, recognized, audio_out, history

    def clear_conversation():
        assistant.conversation_history = []
        return "", "", None, ""

    def get_stats():
        return (
            f"📊 **Session Statistics**\n\n"
            f"Commands Processed: {assistant.session_stats['commands_processed']}\n"
            f"Successful Responses: {assistant.session_stats['successful_responses']}\n"
            f"Errors: {assistant.session_stats['errors']}"
        )

    custom_css = """
    .header-gradient {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 30px;
        border-radius: 15px;
        color: white;
        text-align: center;
        margin-bottom: 25px;
        box-shadow: 0 8px 16px rgba(0,0,0,0.1);
    }
    .feature-card {
        background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
        padding: 15px;
        border-radius: 10px;
        margin: 10px 0;
        border-left: 4px solid #667eea;
        transition: transform 0.2s;
    }
    .feature-card:hover { transform: translateX(5px); }
    .stat-box { background: #e8f4f8; padding: 10px; border-radius: 8px; text-align:center; margin:5px; }
    """

    with gr.Blocks(title="AI Voice Assistant") as demo:
        # inject CSS + header
        gr.HTML(f"""
        <style>
        {custom_css}
        </style>
        <div class="header-gradient">
            <h1 style="margin:0; font-size: 2.5em;">🎙️ Nova Voice Assistant</h1>
            <p style="margin:5px 0 0 0; font-size: 1.1em;">
                Powered by AI • Speech Recognition • Natural Language Processing
            </p>
            <p style="margin:5px 0 0 0; opacity: 0.9;">v2.2 - Professional Edition</p>
        </div>
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### ✨ Key Features")
                gr.HTML("""
                <div class="feature-card"><b>🕐 Time & Date</b><br><small>Get current time and date</small></div>
                <div class="feature-card"><b>🌤️ Weather Updates</b><br><small>Real-time weather for any city</small></div>
                <div class="feature-card"><b>📚 Wikipedia Search</b><br><small>Quick info lookup</small></div>
                <div class="feature-card"><b>🧮 Calculator</b><br><small>Arithmetic with safe evaluation</small></div>
                """)
                stats_btn = gr.Button("📊 View Statistics")
                stats_output = gr.Textbox(label="Statistics", lines=4, visible=False)

            with gr.Column(scale=2):
                gr.Markdown("### 💬 Assistant Interface")
                with gr.Tabs():
                    with gr.Tab("💬 Text Chat"):
                        text_input = gr.Textbox(
                            label="Your Command",
                            placeholder="Type here... (e.g., 'What's the weather in Tokyo?')",
                            lines=2
                        )
                        text_btn = gr.Button("🚀 Send", variant="primary")
                    with gr.Tab("🎤 Voice Input"):
                        gr.Markdown("**Upload an audio file** (WAV recommended). Use your device recorder and upload the file.")
                        audio_input = gr.Audio(label="Upload Audio File", type="filepath")
                        audio_btn = gr.Button("🎵 Process Audio", variant="primary")
                        recognized_output = gr.Textbox(
                            label="Recognized Speech",
                            placeholder="Your spoken words will appear here...",
                            lines=2
                        )

                response_box = gr.Textbox(
                    label="🤖 Assistant Response",
                    lines=6,
                    interactive=False,
                    placeholder="Responses will appear here..."
                )
                audio_response = gr.Audio(label="🔊 Voice Response", autoplay=True)
                with gr.Accordion("📜 Conversation History", open=False):
                    history_box = gr.Textbox(
                        label="Recent Messages",
                        lines=10,
                        interactive=False,
                        placeholder="Your conversation will be saved here..."
                    )
                with gr.Row():
                    clear_btn = gr.Button("🗑️ Clear History")
                    help_btn = gr.Button("❓ Help")

        # Example commands
        gr.Markdown("""
        ---
        ### 🎯 Try These Commands

        | Category | Example Command |
        |----------|----------------|
        | 🎤 Greeting | "Hello Nova" |
        | 🕐 Time | "What time is it?" |
        | 🌤️ Weather | "What's the weather in Tokyo?" |
        | 📚 Knowledge | "Tell me about artificial intelligence" |
        | 🧮 Math | "Calculate 156 divided by 12" |
        | 🔍 Search | "Search for Python tutorials" |
        | ⏰ Reminder | "Remind me to finish the project" |
        """)

        # Handlers
        text_btn.click(
            fn=process_text,
            inputs=[text_input],
            outputs=[response_box, audio_response, history_box]
        )
        text_input.submit(
            fn=process_text,
            inputs=[text_input],
            outputs=[response_box, audio_response, history_box]
        )
        audio_btn.click(
            fn=process_audio,
            inputs=[audio_input],
            outputs=[response_box, recognized_output, audio_response, history_box]
        )
        clear_btn.click(
            fn=clear_conversation,
            outputs=[response_box, recognized_output, audio_response, history_box]
        )
        help_btn.click(
            fn=lambda: (assistant.get_help(), None, ""),
            outputs=[response_box, audio_response, history_box]
        )
        stats_btn.click(
            fn=lambda: gr.update(visible=True, value=get_stats()),
            outputs=[stats_output]
        )

    return demo

# -------------------------
# MAIN
# -------------------------
if __name__ == "__main__":
    print("\n" + "="*60)
    print("🚀 NOVA VOICE ASSISTANT - v2.2 (Fixed Commands + Sanitized TTS)")
    print("="*60 + "\n")
    demo = create_gradio_interface()
    demo.launch(share=True, debug=True, server_name="0.0.0.0")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Selecting previously unselected package libsonic0:amd64.
Preparing to unpack .../libsonic0_0.2.0-11build1_amd64.deb ...
Unpacking libsonic0:amd64 (0.2.0-11build1) ...
Selecting previously unselected package espeak-data:amd64.
Preparing to unpack .../espeak-data_1.48.15+dfsg-3_amd64.deb ...
Unpacking espeak-data:amd64 (1.48.15+dfsg-3) ...
Selecting previously unselected package libespeak1:amd64.
Preparing to unpack .../libespeak1_1.48.15+dfsg-3_amd64.deb ...
Unpacking libespeak1:amd64 (1.48.15+dfsg-3) ...
Selecting previously unselected package espeak.


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://a08a70fd326f2a6196.gradio.live
